In [ ]:
from fastai.vision.all import *
from pathlib import Path
import urllib.request


In [ ]:
from fastai.vision.all import *
from pathlib import Path
import urllib.request
import traceback
import sys

MODEL_URL = "https://github.com/lily-880/Bear-classifier/releases/download/v1/export.pkl"
MODEL_PATH = Path("export.pkl")
TMP_PATH = Path("export.pkl.part")

def download_with_check(url, dst, tmp):
    if tmp.exists(): tmp.unlink()
    print("Downloading model...")
    urllib.request.urlretrieve(url, tmp)

    sz = tmp.stat().st_size
    print("Downloaded bytes:", sz)

    # 打印文件头，判断是不是下载成了 HTML/404 页面
    head = tmp.read_bytes()[:200]
    print("First 200 bytes:", head)

    # 太小基本就不是模型文件
    if sz < 1_000_000:
        raise RuntimeError("Downloaded file too small; likely not the real .pkl (maybe HTML/404).")

    tmp.replace(dst)
    print("Saved to", dst)

try:
    if not MODEL_PATH.exists():
        download_with_check(MODEL_URL, MODEL_PATH, TMP_PATH)
    else:
        print("Model already exists, size:", MODEL_PATH.stat().st_size)

    print("Loading learner...")
    learn = load_learner(MODEL_PATH)
    print("Learner loaded OK.")
except Exception as e:
    print("=== ERROR while downloading/loading model ===")
    traceback.print_exc()
    raise



In [ ]:
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(accept='image/*', multiple=False)
out = widgets.Output()

def on_upload_change(change):
    out.clear_output()
    with out:
        img = PILImage.create(list(uploader.value.values())[0]['content'])
        pred, pred_idx, probs = learn.predict(img)
        display(img.to_thumb(256))
        print(f"Prediction: {pred}")
        print(f"Confidence: {probs[pred_idx]:.3f}")

uploader.observe(on_upload_change, names='value')

display(uploader, out)
